In [1]:
from skilift import GTFS, get_stop_node, read_osm, get_elevations_for_nodes, get_graph_nodes, ElevationAwareStreetDataset

In [2]:
import pandas as pd
from zipfile import ZipFile
from collections import defaultdict
from typing import Dict, Set, Tuple, Iterable, Iterator, Any
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
fn = "/home/bmander/skilift_data/transit/redding.zip"

feed = GTFS(fn)

In [4]:
# get epoch time on Aptil 19, 2023 at 1:00pm pacific time
t0 = pd.Timestamp("2023-04-19 13:00:00", tz="America/Los_Angeles")
t0

Timestamp('2023-04-19 13:00:00-0700', tz='America/Los_Angeles')

In [5]:
node0 = get_stop_node(feed, "Downtown Transit Center", t0)
node0

AtStopNode(stop_id:2000, datetime:2023-04-19 13:00:00-07:00)

In [6]:
node0.outgoing[0].node

DepartureNode(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:0, datetime:2023-04-19 14:25:00-07:00)

In [7]:
node0.outgoing[0].node.outgoing[0].node

ArrivalNode(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:1, datetime:2023-04-19 14:35:00-07:00)

In [8]:
node0.outgoing[0].node.outgoing[0].node.outgoing

[Edge(node=DepartureNode(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:1, datetime:2023-04-19 14:35:00-07:00), weight=0.0),
 Edge(node=AtStopNode(stop_id:8003, datetime:2023-04-19 14:35:00-07:00), weight=60.0)]

In [9]:
osm_filename = "/home/bmander/skilift_data/street/redding.pbf"


In [10]:
elev_geotiff_filename = "/home/bmander/skilift_data/elevation/USGS_13_n41w123_20210624.tif"

In [11]:
elev_dataset = ElevationAwareStreetDataset(osm_filename, elev_geotiff_filename)

Reading OSM file...done
Indexing ways...done
Getting node elevations...done


In [12]:
from shapely.strtree import STRtree
from shapely.geometry import Point, LineString

In [13]:
def geodesic_distance_meters(geo_pt: Point, geo_pt2: Point) -> float:
    """Compute the geodesic distance in meters between two points on the earth's surface.
    """

    # TODO: this can be vectorized if it ever becomes a bottleneck

    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [geo_pt.x, geo_pt.y, geo_pt2.x, geo_pt2.y])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    earth_radius_km = 6371
    km = earth_radius_km * c
    return km * 1000

In [14]:
nd_points = [Point(lon, lat) for lon, lat in elev_dataset.nodes.values()]

In [15]:
tree = STRtree( nd_points )

In [16]:
lon = sum([pt.x for pt in nd_points])/len(nd_points)
lat = sum([pt.y for pt in nd_points])/len(nd_points)
avg = Point(lon, lat)

In [17]:
for nd_ix in tree.query(avg.buffer(0.0003)):
    pt = nd_points[nd_ix]

    # find distance in meters between geographical points
    dist = geodesic_distance_meters(avg, pt)
    print(dist)

33.586155638010396
30.45918994893321
30.448305567346534
30.984222752860745
19.906777976196068


In [18]:
def cons(ary: Iterable) -> Iterator[Tuple[Any, Any]]:
    """Return a generator of consecutive pairs from the input iterable."""
    it = iter(ary)
    prev = next(it)
    for item in it:
        yield prev, item
        prev = item

way_segments = []
for way_id, way in elev_dataset.ways.items():
    for i, (nd1, nd2) in enumerate(cons(way.nds)):
        pt1 = elev_dataset.nodes[nd1]
        pt2 = elev_dataset.nodes[nd2]

        segment = LineString([pt1, pt2])

        way_segments.append( (way_id, i, segment) )


In [19]:
way_segments_index = STRtree([seg[2] for seg in way_segments])

In [20]:
dists = []

for ix in way_segments_index.query(avg.buffer(0.0003)):
    way_id, i, segment = way_segments[ix]

    way_segment_dist = segment.distance(avg)

    dists.append( (ix, way_segment_dist) )

closest_ix = min( dists, key=lambda x: x[1] )[0]

way_id, i, segment = way_segments[closest_ix]
linear_ref = segment.line_locate_point(avg, normalized=True)
print(way_id, i, linear_ref)


10557590 13 0.18715448062975482
